In [42]:
from mip import Model, MAXIMIZE, xsum
from numpy import asarray, abs
import re

class BB:
    def __init__(self):
        self.primal = 0
        self.optimal_model = None

    def ler_arquivo_txt(self, caminho_arquivo: str):
        valores = []
        with open(caminho_arquivo) as arquivo:
            for linha in arquivo:
                for i in re.findall(r'\d+', linha):
                    valores.append(i)

        num_variaveis = int(valores[0])
        num_restricoes = int(valores[1])
        coef_objetivo = []
        for i in range(2, 2 + num_variaveis):
            coef_objetivo.append(int(valores[i]))

        contador = 0
        aux = []
        coef_restricoes = []
        for i in range(2 + num_variaveis, len(valores)):
            if contador == num_variaveis:
                aux.append(int(valores[i]))
                coef_restricoes.append(aux)
                aux = []
                contador = 0
            else:
                aux.append(int(valores[i]))
                contador += 1
        return num_variaveis, num_restricoes, coef_objetivo, coef_restricoes

    def solver(self, modelo: Model):
        modelo.optimize()
        parametros = {}
        parametros["objetivo"] = modelo.objective_value
        parametros["variaveis"] = modelo.vars
        return parametros

    def branch_and_bound(self, modelo: Model):
        nos = [modelo]

        while nos != []:
            modelo_solucao = self.solver(nos[0])
            aux = self.bound(nos[0])
            if aux == 'INVIAVEL' or aux == 'LIMITE':
                nos.pop(0)
            elif aux == 'INTEIRO':
                if modelo_solucao["objetivo"] >= self.primal:
                    self.optimal_model = nos[0]
                    self.primal = modelo_solucao["objetivo"]
                nos.pop(0)
            elif aux == 'VIÁVEL':
                filhos = self.branch(nos[0], modelo_solucao["variaveis"])
                nos.append(filhos[0])
                nos.append(filhos[1])
                nos.pop(0)

    def bound(self, modelo: Model):
        solucao_auxiliar = self.solver(modelo)
        contagem_inteiro = 0

        if solucao_auxiliar["objetivo"] == None:
            return 'INVIAVEL'

        for i in solucao_auxiliar["variaveis"]:
            if i.x.is_integer():
                contagem_inteiro += 1

        if contagem_inteiro == len(solucao_auxiliar["variaveis"]):
            return 'INTEIRO'

        if solucao_auxiliar["objetivo"] <= self.primal:
            return 'LIMITE'
        return 'VIÁVEL'

    def encontrar_mais_proximo(self, array: list, valor: float):
        array = asarray(array)
        idx = (abs(array - valor)).argmin()
        return idx

    def branch(self, modelo: Model, valores_solucao):
        var_ramificacao = valores_solucao[self.encontrar_mais_proximo([i.x for i in valores_solucao], 0.5)]

        modelo_0 = modelo.copy()
        modelo_0 += var_ramificacao == 0

        modelo_1 = modelo.copy()
        modelo_1 += var_ramificacao == 1

        return (modelo_0, modelo_1)

    def imprimir_resultado(self, modelo_resolvido):
        print("Variáveis:")
        for i in modelo_resolvido["variaveis"]:
            print(i.name, ' = ', i.x)
        print("Função objetivo:")
        print('Z = ', modelo_resolvido["objetivo"])

    def executar(self, caminho_arquivo):
        num_variaveis, num_restricoes, coef_objetivo, coef_restricoes = self.ler_arquivo_txt(caminho_arquivo)

        modelo = Model(sense=MAXIMIZE)

        x = [modelo.add_var(var_type="CONTINUOUS",
                            lb=0, ub=1, name="x_" + str(i)) for i in range(num_variaveis)]

        modelo.objective = xsum(coef_objetivo[i] * x[i] for i in range(num_variaveis))

        for i in range(num_restricoes):
            modelo += xsum(coef_restricoes[i][j] * x[j] for j in range(num_variaveis)) <= coef_restricoes[i][-1]

        self.branch_and_bound(modelo)

        self.imprimir_resultado(self.solver(self.optimal_model))


In [44]:
# Instanciar o objeto BB
bb = BB()
bb.executar('teste1.txt')

Variáveis:
x_0  =  0.0
x_1  =  1.0
x_2  =  0.0
x_3  =  0.0
x_4  =  1.0
x_5  =  0.0
x_6  =  0.0
Função objetivo:
Z =  20.0


In [46]:
# Instanciar o objeto BB
bb = BB()
bb.executar('teste2.txt')

Variáveis:
x_0  =  0.0
x_1  =  1.0
x_2  =  0.0
x_3  =  0.0
x_4  =  0.0
x_5  =  1.0
x_6  =  1.0
x_7  =  0.0
x_8  =  0.0
Função objetivo:
Z =  24.0


In [47]:
# Instanciar o objeto BB
bb = BB()
bb.executar('teste3.txt')

Variáveis:
x_0  =  0.0
x_1  =  0.0
x_2  =  1.0
x_3  =  0.0
x_4  =  0.0
x_5  =  0.0
x_6  =  1.0
x_7  =  0.0
x_8  =  0.0
Função objetivo:
Z =  19.0


In [49]:
# Instanciar o objeto BB
bb = BB()
bb.executar('teste4.txt')

Variáveis:
x_0  =  0.0
x_1  =  0.0
x_2  =  1.0
x_3  =  0.0
x_4  =  0.0
x_5  =  0.0
x_6  =  0.0
x_7  =  0.0
x_8  =  0.0
Função objetivo:
Z =  10.0


In [59]:
from mip import Model, MAXIMIZE, xsum
from numpy import asarray, abs
import re

def read_txt(filePath: str):
    l = []
    with open(filePath) as file:
        for line in file:
            for i in re.findall(r'\d+', line):
                l.append(i)

    variables = int(l[0])
    rests = int(l[1])
    coef_obj = []
    for i in range(2, 2 + variables):
        coef_obj.append(int(l[i]))

    count = 0
    aux = []
    coef_rests = []
    for i in range(2 + variables, len(l)):
        if count == variables:
            aux.append(int(l[i]))
            coef_rests.append(aux)
            aux = []
            count = 0
        else:
            aux.append(int(l[i]))
            count += 1
    return variables, rests, coef_obj, coef_rests

def solver(model: Model):
    model.optimize()
    params = {}
    params["objective"] = model.objective_value
    params["variables"] = model.vars
    return params

def bound(model: Model, primal):
    aux_solver = solver(model)
    count_int = 0

    if aux_solver["objective"] == None:
        return 'INVIABLE'

    for i in aux_solver["variables"]:
        if i.x.is_integer():
            count_int += 1

    if count_int == len(aux_solver["variables"]):
        return 'INTEGER'

    if aux_solver["objective"] <= primal:
        return 'LIMIT'
    return 'VIABLE'

def find_nearest(array: list, value: float):
    array = asarray(array)
    idx = (abs(array - value)).argmin()
    return idx

def branch(model: Model, values_solution):
    var_branch = values_solution[find_nearest([i.x for i in values_solution], 0.5)]

    model_0 = model.copy()
    model_0 += var_branch == 0

    model_1 = model.copy()
    model_1 += var_branch == 1

    return (model_0, model_1)

def print_result(model_solved):
    print("Variables:")
    for i in model_solved["variables"]:
        print(i.name, ' = ', i.x)
    print("Objective function:")
    print('Z = ', model_solved["objective"])

def branch_and_bound(model: Model):
    global primal
    global optimal_model
    
    primal = 0
    optimal_model = None
    
    nodes = [model]

    while nodes != []:
        model_solver = solver(nodes[0])
        aux = bound(nodes[0], primal)
        if aux == 'INVIABLE' or aux == 'LIMIT':
            nodes.pop(0)
        elif aux == 'INTEGER':
            if model_solver["objective"] >= primal:
                optimal_model = nodes[0]
                primal = model_solver["objective"]
            nodes.pop(0)
        elif aux == 'VIABLE':
            branches = branch(nodes[0], model_solver["variables"])
            nodes.append(branches[0])
            nodes.append(branches[1])
            nodes.pop(0)

def run(filePath):
    variables, rests, coef_obj, coef_rests = read_txt(filePath)

    model = Model(sense=MAXIMIZE)

    x = [model.add_var(var_type="CONTINUOUS",
                        lb=0, ub=1, name="x_" + str(i)) for i in range(variables)]

    model.objective = xsum(coef_obj[i] * x[i] for i in range(variables))

    for i in range(rests):
        model += xsum(coef_rests[i][j] * x[j] for j in range(variables)) <= coef_rests[i][-1]

    branch_and_bound(model)

    print_result(solver(optimal_model))

# Exemplo de uso:
run("teste1.txt")

Variables:
x_0  =  0.0
x_1  =  1.0
x_2  =  0.0
x_3  =  0.0
x_4  =  1.0
x_5  =  0.0
x_6  =  0.0
Objective function:
Z =  20.0


In [62]:
from mip import Model, MAXIMIZE, xsum
from numpy import asarray, abs
import re

def ler_txt(caminho_arquivo: str):
    l = []
    with open(caminho_arquivo) as arquivo:
        for linha in arquivo:
            for i in re.findall(r'\d+', linha):
                l.append(i)

    variaveis = int(l[0])
    restricoes = int(l[1])
    coef_obj = []
    for i in range(2, 2 + variaveis):
        coef_obj.append(int(l[i]))

    count = 0
    aux = []
    coef_restricoes = []
    for i in range(2 + variaveis, len(l)):
        if count == variaveis:
            aux.append(int(l[i]))
            coef_restricoes.append(aux)
            aux = []
            count = 0
        else:
            aux.append(int(l[i]))
            count += 1
    return variaveis, restricoes, coef_obj, coef_restricoes

def resolver(modelo: Model):
    modelo.optimize()
    parametros = {}
    parametros["objetivo"] = modelo.objective_value
    parametros["variaveis"] = modelo.vars
    return parametros

def limite(modelo: Model, primal):
    aux_solver = resolver(modelo)
    count_int = 0

    if aux_solver["objetivo"] == None:
        return 'INVIAVEL'

    for i in aux_solver["variaveis"]:
        if i.x.is_integer():
            count_int += 1

    if count_int == len(aux_solver["variaveis"]):
        return 'INTEIRO'

    if aux_solver["objetivo"] <= primal:
        return 'LIMITANTE'
    return 'VIÁVEL'

def encontrar_mais_proximo(array: list, valor: float):
    array = asarray(array)
    idx = (abs(array - valor)).argmin()
    return idx

def ramificar(modelo: Model, valores_solucao):
    var_ramificacao = valores_solucao[encontrar_mais_proximo([i.x for i in valores_solucao], 0.5)]

    modelo_0 = modelo.copy()
    modelo_0 += var_ramificacao == 0

    modelo_1 = modelo.copy()
    modelo_1 += var_ramificacao == 1

    return (modelo_0, modelo_1)

def imprimir_resultado(modelo_resolvido):
    print("Variáveis:")
    for i in modelo_resolvido["variaveis"]:
        print(i.name, ' = ', i.x)
    print("Função objetivo:")
    print('Z = ', modelo_resolvido["objetivo"])

def branch_and_bound(modelo: Model):
    global primal
    global modelo_otimo
    
    primal = 0
    modelo_otimo = None
    
    nos = [modelo]

    while nos != []:
        modelo_solver = resolver(nos[0])
        aux = limite(nos[0], primal)
        if aux == 'INVIAVEL' or aux == 'LIMITANTE':
            nos.pop(0)
        elif aux == 'INTEIRO':
            if modelo_solver["objetivo"] >= primal:
                modelo_otimo = nos[0]
                primal = modelo_solver["objetivo"]
            nos.pop(0)
        elif aux == 'VIÁVEL':
            ramos = ramificar(nos[0], modelo_solver["variaveis"])
            nos.append(ramos[0])
            nos.append(ramos[1])
            nos.pop(0)

def executar(caminho_arquivo):
    variaveis, restricoes, coef_obj, coef_restricoes = ler_txt(caminho_arquivo)

    modelo = Model(sense=MAXIMIZE)

    x = [modelo.add_var(var_type="CONTINUOUS", lb=0, ub=1, name="x_" + str(i)) for i in range(variaveis)]

    modelo.objective = xsum(coef_obj[i] * x[i] for i in range(variaveis))

    for i in range(restricoes):
        modelo += xsum(coef_restricoes[i][j] * x[j] for j in range(variaveis)) <= coef_restricoes[i][-1]

    branch_and_bound(modelo)

    imprimir_resultado(resolver(modelo_otimo))

# Exemplo de uso:
executar("teste1.txt")

Variáveis:
x_0  =  0.0
x_1  =  1.0
x_2  =  0.0
x_3  =  0.0
x_4  =  1.0
x_5  =  0.0
x_6  =  0.0
Função objetivo:
Z =  20.0


Exemplo teste1:

$$
\text{max } Z = 2x_1 + 10x_2 + 8x_3 + 7x_4 + 10x_5 + 10x_6 + 6x_7
$$

$$
\begin{gather*}
5x_1 + 7x_2 + 8x_3 + 1x_4 + 7x_5 + 5x_6 + 6x_7\leq 20 \\
1x_1 + 6x_2 + 4x_3 + 9x_4 + 10x_5 + 6x_6 + 10x_7 \leq 30 \\
4x_1 + 4x_2 + 4x_3 + 1x_4 + 5x_5 + 5x_6 + 10x_7 \leq 40 \\
3x_1 + 10x_2 + 8x_3 + 1x_4 + 3x_5 + 3x_6 + 8x_7\leq 30 \\
10x_1 + 8x_2 + 9x_3 + 9x_4 + 7x_5 + 6x_6 + 10x_7\leq 20 \\
6x_1 + 6x_2 + 3x_3 + 6x_4 + 3x_5 + 7x_6 + 2x_7\leq 80 \\
7x_1 + 10x_2 + 7x_3 + 8x_4 + 7x_5 + 8x_6 + 7x_7\leq 100 \\
9x_1 + 8x_2 + 1x_3 + 1x_4 + 8x_5 + 10x_6 + 2x_7\leq 90 \\
1x_1 + 5x_2 + 3x_3 + 10x_4 + 2x_5 + 4x_6 + 9x_7\leq 70 \\
9x_1 + 6x_2 + 1x_3 + 4x_4 + 7x_5 + 5x_6 + 10x_7\leq 60 \\
5x_1 + 7x_2 + 4x_3 + 4x_4 + 3x_5 + 4x_6 + 10x_7\leq 40 \\
x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0 \\
\end{gather*}
$$
<div style="text-align: center; color: red;"><s>$x_1, x_2, x_3, x_4, x_5, x_6, x_7 \in \{0, 1\}$</s></div>


s.a. 
$$
\begin{gather*}
5x_1 + 7x_2 + 8x_3 + 1x_4 + 7x_5 + 5x_6 + 6x_7\leq 20 \\
1x_1 + 6x_2 + 4x_3 + 9x_4 + 10x_5 + 6x_6 + 10x_7 \leq 30 \\
4x_1 + 4x_2 + 4x_3 + 1x_4 + 5x_5 + 5x_6 + 10x_7 \leq 40 \\
3x_1 + 10x_2 + 8x_3 + 1x_4 + 3x_5 + 3x_6 + 8x_7\leq 30 \\
10x_1 + 8x_2 + 9x_3 + 9x_4 + 7x_5 + 6x_6 + 10x_7\leq 20 \\
6x_1 + 6x_2 + 3x_3 + 6x_4 + 3x_5 + 7x_6 + 2x_7\leq 80 \\
7x_1 + 10x_2 + 7x_3 + 8x_4 + 7x_5 + 8x_6 + 7x_7\leq 100 \\
9x_1 + 8x_2 + 1x_3 + 1x_4 + 8x_5 + 10x_6 + 2x_7\leq 90 \\
1x_1 + 5x_2 + 3x_3 + 10x_4 + 2x_5 + 4x_6 + 9x_7\leq 70 \\
9x_1 + 6x_2 + 1x_3 + 4x_4 + 7x_5 + 5x_6 + 10x_7\leq 60 \\
5x_1 + 7x_2 + 4x_3 + 4x_4 + 3x_5 + 4x_6 + 10x_7\leq 40 \\
x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0 \\
x_1, x_2, x_3, x_4, x_5, x_6, x_7 \in \{0, 1\}
\end{gather*}
$$